# General data cleaning techniques

The following examples show application of general data cleaning techniques. By general techniques we mean the current techniques applied. Missing data and outliers are excluded from these general techniques, as they are more problem-dependent.

General data cleaning techniques:
1. [Data types](#Data-types)
1. [Data standardization](#Data-standardization)
1. [Constant features](#Constant-features)
1. [Duplicated rows](#Duplicated-rows)
1. [Duplicated features](#Duplicated-features)
1. [Values out of range](#Values-out-of-range)
1. [Shuffle dataset](#Shuffle-dataset)

For further information, see the corresponding blog [post]().

---

# Data types

Let's use the Kaggle Titanic dataset as an example.

In [17]:
# Import data 
import pandas as pd
df = pd.read_csv('./data/titanic.csv')  # Kaggle Titanic dataset

# Corrupt dataset to make the example meaningful
df['Age'] = df['Age'].astype('object')

# Show dataset
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


If you need to remind the meaning of each feature, check this [link](https://www.kaggle.com/pmarcelino/data-analysis-and-feature-extraction-with-python).

In [18]:
# Get list of features
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

What we expect to see when checking features' data types:

| Feature     | Data type        | Comment                                                    |
|-------------|------------------|------------------------------------------------------------|
| PassengerId | int64            | It's a categorical feature.                                |
| Survived    | int64            | It's a categorical feature.                                |
| Pclass      | int64            | It's an ordinal feature.                                   |
| Name        | object           | It's character data (string).                              |
| Sex         | int64 or object  | Categorical feature. It can be [0,1] or ['male','female']. |
| Age         | int64 or float64 | float64 if age = date of birth - current date.             |
| SibSp       | int64            | It's continuous.                                           |
| Parch       | int64            | It's continuous.                                           |
| Ticket      | object or int64  | It's categorical.                                          |
| Fare        | float64          | It's continuous.                                           |
| Cabin       | int64            | It's categorical.                                          |
| Embarked    | int64 or object  | It's categorical.                                          |

Looks cool, isn't it? If you also want to do fancy tables, use [this](https://www.tablesgenerator.com/markdown_tables#).

In [19]:
# Get data types
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age             object
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

The only feature that it's not according to our expectation is 'Age' (what a surprise). This feature is defined as character data (object), when it should be numeric data (int64 or float64).

We can use [pandas.to_numeric](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html#pandas.to_numeric) to correct this situation.

In [20]:
# Correct data type
df['Age'] = pd.to_numeric(df['Age'])

# Debug
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Clean! This is the type of data cleaning operations related to data types that we usually do.

Other common data type transformations are:

* [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html#pandas.to_datetime) - Converts argument to datetime.
* [pandas.to_timedelta](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_timedelta.html#pandas.to_timedelta) - Converts argument to timedelta.
* [pandas.DataFrame.astype('object')](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html#pandas.DataFrame.astype) - Converts argument to object. This is a generic transformation. You can use it to transform data to other data types than object.

[Go back to the top](#General-data-cleaning-techniques)

---

# Data standardization

<img src=https://i.imgflip.com/2f4wfc.jpg>

---

# Constant features

In this case, we will use data from the [Santander Customer Satisfaction](https://www.kaggle.com/c/santander-customer-satisfaction/data) competition.

In [21]:
# Import data
import pandas as pd

df = pd.read_csv('./data/santander.csv')
df.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,ind_var14_0,ind_var14,ind_var17_0,ind_var17,ind_var18_0,ind_var18,ind_var19,ind_var20_0,ind_var20,ind_var24_0,ind_var24,ind_var25_cte,ind_var26_0,ind_var26_cte,ind_var26,ind_var25_0,ind_var25,ind_var27_0,ind_var28_0,ind_var28,ind_var27,ind_var29_0,ind_var29,ind_var30_0,ind_var30,ind_var31_0,ind_var31,ind_var32_cte,ind_var32_0,ind_var32,ind_var33_0,ind_var33,ind_var34_0,ind_var34,ind_var37_cte,ind_var37_0,ind_var37,ind_var39_0,ind_var40_0,ind_var40,ind_var41_0,ind_var41,ind_var39,ind_var44_0,ind_var44,ind_var46_0,ind_var46,num_var1_0,num_var1,num_var4,num_var5_0,num_var5,num_var6_0,num_var6,num_var8_0,num_var8,num_var12_0,num_var12,num_var13_0,num_var13_corto_0,num_var13_corto,num_var13_largo_0,num_var13_largo,num_var13_medio_0,num_var13_medio,num_var13,num_var14_0,num_var14,num_var17_0,num_var17,num_var18_0,num_var18,num_var20_0,num_var20,num_var24_0,num_var24,num_var26_0,num_var26,num_var25_0,num_var25,num_op_var40_hace2,num_op_var40_hace3,num_op_var40_ult1,num_op_var40_ult3,num_op_var41_hace2,num_op_var41_hace3,num_op_var41_ult1,num_op_var41_ult3,num_op_var39_hace2,num_op_var39_hace3,num_op_var39_ult1,num_op_var39_ult3,num_var27_0,num_var28_0,num_var28,num_var27,num_var29_0,num_var29,num_var30_0,num_var30,num_var31_0,num_var31,num_var32_0,num_var32,num_var33_0,num_var33,num_var34_0,num_var34,num_var35,num_var37_med_ult2,num_var37_0,num_var37,num_var39_0,num_var40_0,num_var40,num_var41_0,num_var41,num_var39,num_var42_0,num_var42,num_var44_0,num_var44,num_var46_0,num_var46,saldo_var1,saldo_var5,saldo_var6,saldo_var8,saldo_var12,saldo_var13_corto,saldo_var13_largo,saldo_var13_medio,saldo_var13,saldo_var14,saldo_var17,saldo_var18,saldo_var20,saldo_var24,saldo_var26,saldo_var25,saldo_var28,saldo_var27,saldo_var29,saldo_var30,saldo_var31,saldo_var32,saldo_var33,saldo_var34,saldo_var37,saldo_var40,saldo_var41,saldo_var42,saldo_var44,saldo_var46,var36,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,delta_imp_trasp_var17_out_1y3,delta_imp_trasp_var33_in_1y3,delta_imp_trasp_var33_out_1y3,delta_imp_venta_var44_1y3,delta_num_aport_var13_1y3,delta_num_aport_var17_1y3,delta_num_aport_var33_1y3,delta_num_compra_var44_1y3,delta_num_reemb_var13_1y3,delta_num_reemb_var17_1y3,delta_num_reemb_var33_1y3,delta_num_trasp_var17_in_1y3,delta_num_trasp_var17_out_1y3,delta_num_trasp_var33_in_1y3,delta_num_trasp_var33_out_1y3,delta_num_venta_var44_1y3,imp_amort_var18_hace3,imp_amort_var18_ult1,imp_amort_var34_hace3,imp_amort_var34_ult1,imp_aport_var13_hace3,imp_aport_var13_ult1,imp_aport_var17_hace3,imp_aport_var17_ult1,imp_aport_var33_hace3,imp_aport_var33_ult1,imp_var7_emit_ult1,imp_var7_recib_ult1,imp_compra_var44_hace3,imp_compra_var44_ult1,imp_reemb_var13_hace3,imp_reemb_var13_ult1,imp_reemb_var17_hace3,imp_reemb_var17_ult1,imp_reemb_var33_hace3,imp_reemb_var33_ult1,imp_var43_emit_ult1,imp_trans_var37_ult1,imp_trasp_var17_in_hace3,imp_trasp_var17_in_ult1,imp_trasp_var17_out_hace3,imp_trasp_var17_out_ult1,imp_trasp_var33_in_hace3,imp_trasp_var33_in_ult1,imp_trasp_var33_out_hace3,imp_trasp_var33_out_ult1,imp_venta_var44_hace3,imp_venta_var44_ult1,ind_var7_emit_ult1,ind_var7_recib_ult1,ind_var10_ult1,ind_var10

Constant features are features that have always the same value. 

A quick way to inspect this is to look for features with null standard deviation. This method works for numeric features. Object features follow a similar approach, but there are some differences that we will discuss later.

In [22]:
# Get list of possible constant features 
s = df.describe().loc['std']  # Get standard deviation of each feature
candidates = s.where(s==0).dropna().index  # Get list of features with null standard deviation

In [23]:
# Inspect features
pd.set_option('display.max_columns', 500)  # Change definitions to show all columns
df[candidates].describe()

,ind_var2_0,ind_var2,ind_var27_0,ind_var28_0,ind_var28,ind_var27,ind_var41,ind_var46_0,ind_var46,num_var27_0,num_var28_0,num_var28,num_var27,num_var41,num_var46_0,num_var46,saldo_var28,saldo_var27,saldo_var41,saldo_var46,imp_amort_var18_hace3,imp_amort_var34_hace3,imp_reemb_var13_hace3,imp_reemb_var33_hace3,imp_trasp_var17_out_hace3,imp_trasp_var33_out_hace3,num_var2_0_ult1,num_var2_ult1,num_reemb_var13_hace3,num_reemb_var33_hace3,num_trasp_var17_out_hace3,num_trasp_var33_out_hace3,saldo_var2_ult1,saldo_medio_var13_medio_hace3
count,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0,76020.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


All features are constant. They have always the same value, which is zero in this case. Let's delete these features.

In [24]:
# Delete features
df.drop(candidates, axis=1, inplace=True)

In [25]:
# Test 
candidates.isin(df.columns)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

As we can see, the features that we identified as constant are no longer in the dataset. The dataset is now clean from numeric constant features.

What about object features? Let's see.

In [26]:
# Get character features
df.dtypes.where(df.dtypes=='object').dropna()

Series([], dtype: object)

In this case, we don't have object features. If we had, we would have to describe them and search for those with only one value.

Let's create a fake dataset to exemplify this.

In [27]:
# Create example 
df = pd.DataFrame({'player':['eusebio', 'jonas', 'magnusson', 'nene'],
                  'team':['benfica', 'benfica', 'benfica', 'benfica']})
df

,player,team
0,eusebio,benfica
1,jonas,benfica
2,magnusson,benfica
3,nene,benfica


In [28]:
# Search for constant features
s = df.describe().loc['unique']  # Look for the number of unique values in each feature
candidates = s.where(s==1).dropna().index  # Get list of features with only one unique value
candidates

Index(['team'], dtype='object')

In this case, the *team* feature is the only constant features. All the observations have the same *team* value (benfica).

In [29]:
# Delete constant features
df.drop(candidates, axis=1, inplace=True)
df

,player
0,eusebio
1,jonas
2,magnusson
3,nene


Clean! Now we only have those non-constant features.

**Note:** The *.describe()* structure of this dataset is different from the one we saw on the Santander dataset. That happens because now we are dealing with object features and previously we were dealing with numeric features. You can read more on how to describe datasets with different data types [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html).

[Go back to the top](#General-data-cleaning-techniques)

---

# Duplicated rows

And now for something completely different, a Benfica dataset.

In [30]:
# Create example 
df = pd.DataFrame({'favorite_team':['benfica', 'olhanense', 'benfica', 'benfica', 'benfica'],
                   'player':['eusebio', 'djalmir', 'magnusson', 'nene', 'eusebio'],
                   'team':['benfica', 'olhanense', 'benfica', 'benfica','benfica'],
                   'goals':[102,44,78,65,102]})
df

,favorite_team,goals,player,team
0,benfica,102,eusebio,benfica
1,olhanense,44,djalmir,olhanense
2,benfica,78,magnusson,benfica
3,benfica,65,nene,benfica
4,benfica,102,eusebio,benfica


To delete the duplicated rows, we just need to apply [pandas.DataFrame.drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html).

In [31]:
# Delete duplicated rows
df.drop_duplicates(df, keep='last', inplace=True)
df

,favorite_team,goals,player,team
1,olhanense,44,djalmir,olhanense
2,benfica,78,magnusson,benfica
3,benfica,65,nene,benfica
4,benfica,102,eusebio,benfica


Note that it deleted row 0. That happened because I chose to keep the last duplicated observation. If I had chosen to keep the first duplicated observation, then row 4 would have been deleted instead of row 0.

[Go back to the top](#General-data-cleaning-techniques)

---

# Duplicated features

Let's extend the previous example.

In [32]:
# Create example 
df = pd.DataFrame({'favorite_team':['benfica', 'olhanense', 'benfica', 'benfica'],
                   'player':['eusebio', 'djalmir', 'magnusson', 'nene'],
                   'team':['benfica', 'olhanense', 'benfica', 'benfica'],
                   'goals':[102,44,78,65]})
df

,favorite_team,goals,player,team
0,benfica,102,eusebio,benfica
1,olhanense,44,djalmir,olhanense
2,benfica,78,magnusson,benfica
3,benfica,65,nene,benfica


As we can see, our dataset has duplicated features (*favorite_team* and *team*). In this case, we can't apply *drop_duplicates* directly because it only works to detect duplicated rows. However, there's a smart workaround.

In [33]:
# Delete duplicated features
temp = df.T.drop_duplicates()  # Transform columns into rows
df = temp.T
df

,favorite_team,goals,player
0,benfica,102,eusebio
1,olhanense,44,djalmir
2,benfica,78,magnusson
3,benfica,65,nene


Voilà! Now the dataset is clean.

[Go back to the top](#General-data-cleaning-techniques)

---

# Values out of range

In this section, we only deal with those values that are clearly wrong because they are impossible to happen. For example, no one can have a negative age (not even Benjamin Button). There is a different type of values out of range, which belongs to the outliers analysis. Those values correspond to situations that are anomalous, but not necessarily impossible. We will analyse outliers in a different notebook.

For now, let's check what's going on in our favorite dataset.

In [20]:
# Create example 
df = pd.DataFrame({'player':['eusebio', 'magnusson', 'nene'],
                   'team':['benfica', 'benfica', 'benfica'],
                   'age':[71,55,-68]})
df

,age,player,team
0,71,eusebio,benfica
1,55,magnusson,benfica
2,-68,nene,benfica


In a real case, I'd probably detect that Nené's age is wrong through one of two processes:
1. Impose up front that all age values must be above 0 (kind of sanity check).
1. Check minimum and maximum values in the descriptive statistics.

Let's see how to proceed in each case.

### Sanity check

In [21]:
# Impose condition for age values
df_sane = df[df['age']>0]
df_sane

,age,player,team
0,71,eusebio,benfica
1,55,magnusson,benfica


### Descriptive statistics

In [22]:
# Case 2
df.describe()

,age
count,3.000000
mean,19.333333
std,76.054805
min,-68.000000
25%,-6.500000
50%,55.000000
75%,63.000000
max,71.000000


One of the things that I pay attention when analysing descriptive statistics is *min* and *max* values. It's one of the best ways I know to catch those gross errors that would emabarass you in front of your boss.

In this case, the value -68 would get my attention and I'd remove it.

In [25]:
# See what's going on
df[df['age']==df['age'].min()]

,age,player,team
2,-68,nene,benfica


In [26]:
# Drop row
df = df[df['age']>df['age'].min()]
df

,age,player,team
0,71,eusebio,benfica
1,55,magnusson,benfica


It's good practice to check descriptive statistics again. We never know what else is hidding there.

In [27]:
# Descriptive statistics (take 2)
df.describe()

,age
count,2.000000
mean,63.000000
std,11.313708
min,55.000000
25%,59.000000
50%,63.000000
75%,67.000000
max,71.000000


Looks nice. Mission accomplished!

[Go back to the top](#General-data-cleaning-techniques)

---

# Shuffle dataset

Datasets should be shuffled before training machine learning models to avoid bias/patterns situations. It's easy to forget about this, but in many cases datasets are ordered in such a way that can bias your analysis.

Consider the following example:

In [45]:
# Create example 
df = pd.DataFrame({'name':['pussidonio', 'benquerenca', 'jorinho','balsagodes','asdrubal','tamagnini'],
                   'salary':['780', '767', '750','760','2580','2750'],
                   'age':[18,19,20,19,56,58]})
df

,age,name,salary
0,18,pussidonio,780
1,19,benquerenca,767
2,20,jorinho,750
3,19,balsagodes,760
4,56,asdrubal,2580
5,58,tamagnini,2750


Now imagine that you have the following train and test datasets.

In [46]:
# Create train and test datasets
import numpy as np

train_size = 0.8
n_train = int(np.shape(df)[0] * train_size)

df_train = df.iloc[:n_train]
df_test = df.iloc[n_train:]

If you create such datasets, you're in trouble. Comparing both datasets, we can see that the distribution of values is completely different. While in the train set age is between 18-20 and salaries are around 764, in the test set age is between 56-58 and salaries around 2665. Accordingly, using our train set we can't learn much about our test set.

The message here is that we should shuffle our dataset to avoid this type of segregation, which bias our analysis. There are two ways to do so: 
1. Directly shuffle the dataset.
1. Create train and test sets with sklearn-learn.

### Directly shuffle the dataset

In [47]:
# Shuffle dataset
from sklearn.utils import shuffle

df_shuffled = shuffle(df)
df_shuffled

,age,name,salary
0,18,pussidonio,780
3,19,balsagodes,760
4,56,asdrubal,2580
1,19,benquerenca,767
2,20,jorinho,750
5,58,tamagnini,2750


Now the data is mixed. Note that we can still have [imbalanced datasets](https://blog.dominodatalab.com/imbalanced-datasets/), but that's a different problem. 

### Create train and test sets with scikit-learn

In [51]:
from sklearn.model_selection import train_test_split

X = df.drop('salary', axis=1)
y = df['salary']

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)
print('X_train:\n', X_train)
print('X_test:\n', X_test)

X_train:
    age        name
5   58   tamagnini
3   19  balsagodes
2   20     jorinho
4   56    asdrubal
X_test:
    age         name
1   19  benquerenca
0   18   pussidonio


Same comment as before: data is mixed but we can have imbalanced datasets.

[Go back to the top](#General-data-cleaning-techniques)